In [9]:
from datetime import date
from os.path import basename, splitext
import re

from pyspark.sql import Row
from pyspark.sql.types import *


# TODO figure out why when scanning all records grep got a count of
# 120924 but the data frame has 120700.
# TODO make a SQL comment for dataframes for defaulted ints (like -1)


In [10]:
# retroFilePath = "retrosheet-data/*/*.EV[AN]"
retroFilePath = "retrosheet-data/2010*/*.EV[AN]" # just the 10s

rosterPath = "retrosheet-data/*/*.ROS"
teamPath = "retrosheet-data/*/TEAM[0-9][0-9][0-9][0-9]"

DIST_PARQUET_DIR = "dist/parquet/"


In [46]:

# EVENT FILES: the following is generic stuff to processes
# the event files, ie *.EV[AN]


# proc to process a whole game's events. 
# by using '\nid,' as our delim above, we get called with a whole game's
# events. we resplit them on newlines so lines can be parsed with event type
# specific parsers, but first prepend the game_id and a sequence into the game's
# events to each record.
def processRecord( (k,recordsBetweenIdLines) ):

    # first, resplit the events
    events = recordsBetweenIdLines.splitlines()

    # first off, disgard non-game related records, ie they dont appear 
    # after an "id," token. examples:
    # 
    # com,"Copyright 2001 by Stats Inc."
    # com,"All Rights Reserved."
    if (events[0].startswith("com,")):
        return []
    
    # if we're not the first record, we dont still have the "id," token
    # restore the record so the id record parser can work like the others
    
    if (events[0].startswith("id,") is False):
        events[0] = "id," + events[0]
        
    # now, get the game_id that will be prepended to each record
    game_id = events[0].split(",")[1]    
    homeTeamCode = game_id[:3]
    gameYear = int(game_id[3:7])
    gameMonth = int(game_id[7:9])
    gameDay = int(game_id[9:11])
    gameDate = date(gameYear, gameMonth, gameDay)

    # now emit lines prepended with game_id and the seq into the game, then
    # the original lines from the file.
    return (( 
        game_id, 
        seq,
        homeTeamCode,
        gameDate,
        record.split(",") ## TODO consider flattening this in
    ) for seq, record in enumerate(events))

## some stuff to hide details of the global columns prepended 
## to each event in processRecord() so the mappers below dont
## need to know about them

BASE_GLOBAL_COLUMNS_SCHEMA = [    
    StructField("game_id",StringType(),False), 
    StructField("seq",ShortType(),False), # TODO should be short
    StructField("homeTeamCode",StringType(),False), 
    StructField("gameDate",DateType(),False), 
]
def getBaseColumns(record):
    return [
        record[0],
        int(record[1]),
        record[2],
        record[3]
    ]
def getFields(record): 
    return record[-1]


#### Here is where the processing game event processing begins

# first, make an RDD that has all of a games records combined

retrosheet = sc.newAPIHadoopFile( 
    retroFilePath, 
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat', 
    'org.apache.hadoop.io.LongWritable', 
    'org.apache.hadoop.io.Text', 
    conf={
        'textinputformat.record.delimiter':'\nid,'
    }
)

# then, do a flatmap to get the events between "id" records and to 
# process them using the proc above
combinedFlattenedEvents = retrosheet.flatMap(processRecord)
combinedFlattenedEvents


PythonRDD[269] at RDD at PythonRDD.scala:43

In [61]:

## EVENT TYPE SPECIFIC STUFF

## TODO get consistent with underscore vs camelCase for columns
SCHEMA_BY_TYPE = {    
    
    # sample line: start,howar001,"Ryan Howard",0,4,3               
    # sample line: sub,waldj001,"Jordan Walden",0,0,1
    'start': { 
        'schema': StructType(BASE_GLOBAL_COLUMNS_SCHEMA + [
            StructField("player_id",StringType(),False), 
            StructField("playerName",StringType(),False), 
            StructField("home",BooleanType(),False), 
            StructField("battingOrder",ShortType(),False), ## TODO can this be short
            StructField("position",ShortType(),False)
        ]), 
        'mapper': lambda rec: getBaseColumns(rec) + [
            getFields(rec)[1],
            getFields(rec)[2],
            ("0" == getFields(rec)[3]),  ## TODO verify "0" is home
            int(getFields(rec)[4]),
            int(re.sub('[^0-9]','',getFields(rec)[5])) # make this a reusable proc            
        ]
    },  
    
    # sample line: play,6,1,bondb001,02,CFX,HR/9.3-H;2-H;1-H  
    'play': { 
        'schema': StructType(BASE_GLOBAL_COLUMNS_SCHEMA + [ 

            StructField("inning",ShortType(),False), 
            StructField("topOfInning",BooleanType(),False), 
            StructField("player_id",StringType(),False), 
            StructField("count",ArrayType(ShortType(), False),True), 
            StructField("pitch_seq",StringType(),False),
            StructField("description",StringType(),False)
        ]),  
        'mapper': lambda rec: getBaseColumns(rec) + [
            int(getFields(rec)[1]),             
            bool("0" == getFields(rec)[2]),
            getFields(rec)[3],
            [int(c) for c in list(getFields(rec)[4])] if getFields(rec)[4].isdigit() else [],
            getFields(rec)[5],
            getFields(rec)[6]            
        ]
        
    },    
    
    # sample line: 
    # com,"$Career homer 587 to pass Frank Robinson for 4th all-time"
    
    # or sample multi-line comment:
    # com,"$Hall caught in rundown while Winn advanced to 3B; both players"
    # com,"ended up on 3B and Winn is tagged out; Hall thought he was the one"
    # com,"who was out and stepped off the bag and is tagged out"  
    ## TODO collapse all of these com records should be one comment
    'com': { 
        'schema': StructType(BASE_GLOBAL_COLUMNS_SCHEMA + [  
            StructField("comment",StringType(),False)
        ]), 
        'mapper': lambda rec: getBaseColumns(rec) + [
            getFields(rec)[1]
        ]
    },
    
    
    # sample line: info,attendance,41128
    'info': { 
        'schema': StructType(BASE_GLOBAL_COLUMNS_SCHEMA + [ 
            StructField("key",StringType(),False), 
            StructField("value",StringType(),False)
        ]), 
        'mapper': lambda rec: getBaseColumns(rec) + getFields(rec)[1:]
    },
    
    # sample line: data,er,fyhrm001,0
    # sample line: padj,harrg001,L
    # sample line: ladj,0,9
    # sample line: badj,bonib001,R
    
    # for these, we're generic. just dump the record. store this for data
    # but we clone it below for the other types
    'data': { 
        'schema': StructType(BASE_GLOBAL_COLUMNS_SCHEMA + [
            StructField("type",StringType(),False), 
            StructField("key",StringType(),False), 
            StructField("value",StringType(),False)
        ]), 
        'mapper': lambda rec: getBaseColumns(rec) + getFields(rec) # 
    },   
    
}

## cloned record types for similar layouts
SCHEMA_BY_TYPE['padj'] = SCHEMA_BY_TYPE['data']
SCHEMA_BY_TYPE['badj'] = SCHEMA_BY_TYPE['data']
SCHEMA_BY_TYPE['ladj'] = SCHEMA_BY_TYPE['data']

SCHEMA_BY_TYPE['sub'] = SCHEMA_BY_TYPE['start']

testTypes = {}
for t in ['info','padj','badj','ladj']:
    testTypes[t] = SCHEMA_BY_TYPE[t]

dfByType = {}
# for recType,typeInfo in SCHEMA_BY_TYPE.iteritems():
for recType,typeInfo in testTypes.iteritems():    
    curRdd = combinedFlattenedEvents.filter(lambda rec: rec[-1][0]==recType)
    if curRdd.isEmpty(): continue 
    dfByType[recType] = sqlContext.createDataFrame(
        curRdd.map(typeInfo['mapper']), typeInfo['schema']
    )
    print("%s has %d records" % (recType,dfByType[recType].count()))
    dfByType[recType].printSchema()
    dfByType[recType].show()


print("done")


info has 364477 records
root
 |-- game_id: string (nullable = false)
 |-- seq: short (nullable = false)
 |-- homeTeamCode: string (nullable = false)
 |-- gameDate: date (nullable = false)
 |-- key: string (nullable = false)
 |-- value: string (nullable = false)

+------------+---+------------+----------+---------+----------+
|     game_id|seq|homeTeamCode|  gameDate|      key|     value|
+------------+---+------------+----------+---------+----------+
|ANA201004050|  2|         ANA|2010-04-05|  visteam|       MIN|
|ANA201004050|  3|         ANA|2010-04-05| hometeam|       ANA|
|ANA201004050|  4|         ANA|2010-04-05|     site|     ANA01|
|ANA201004050|  5|         ANA|2010-04-05|     date|2010/04/05|
|ANA201004050|  6|         ANA|2010-04-05|   number|         0|
|ANA201004050|  7|         ANA|2010-04-05|starttime|    7:08PM|
|ANA201004050|  8|         ANA|2010-04-05| daynight|     night|
|ANA201004050|  9|         ANA|2010-04-05|    usedh|      true|
|ANA201004050| 10|         ANA|20

In [6]:
# the following processes the roster files, ie *.ROS, and team files 
# TEAMyyyy. it prepends the rows in the file with its filename to extract
# things like the year

## this proc does two things, first it is a map function for a wholeTextFiles() call. It is
## meant to be called with a flatmap because it returns the rows of the file with the last part
## of the filename preprended to the records from the file. Additionally, this method can do a
## transforms on the file part which is passed in with transformPath parameter.
def getWholeTextParserWithPathForOffset(sliceFrom=None, sliceTo=None, cast=lambda x:x):
    def wholeTextParserWithPath( (path, content) ): 
        fileName = splitext(basename(path))[0]
        return [ [cast(fileName[sliceFrom:sliceTo])] + record.split(",") for record in content.splitlines()]
    return wholeTextParserWithPath


## this proc does two things, first it is a map function for a wholeTextFiles() call. It is
## meant to be called with a flatmap because it returns the rows of the file with the last part
## of the filename preprended to the records from the file. Additionally, this method can do a
## transforms on the file part which is passed in with the optional pathTransform parameter. this
## defaults to just an identity function but can be used to trim the path and cast to an int, say.
def getWholeTextParserWithPathTransform(pathTransform=lambda x:x):
    def wholeTextParserWithPath( (path, content) ): 
        fileName = splitext(basename(path))[0]
        return [ [pathTransform(fileName)] + record.split(",") for record in content.splitlines()]
    return wholeTextParserWithPath



### ROSTER files
# sample greiz001,Greinke,Zack,R,R,LAN,P
rosterSchema = StructType([
    # prepended from filename
    StructField("year",ShortType(),False), 

    # rest from record
    StructField("player_id",StringType(),False), 
    StructField("firstName",StringType(),False), 
    StructField("lastName",StringType(),False), 
    StructField("bats",StringType(),False), 
    StructField("throws",StringType(),False), 
    StructField("team",StringType(),False), 
    StructField("position",StringType(),False)
])

rosterRowsRDD = sc.wholeTextFiles(rosterPath).flatMap(
    getWholeTextParserWithPathTransform(lambda x:int(x[3:])) # trim year part and cast to int
)
rosterDataFrame = sqlContext.createDataFrame(rosterRowsRDD,rosterSchema)
rosterDataFrame.registerTempTable("roster")
rosterDataFrame.write.save(DIST_PARQUET_DIR + "roster", format="parquet", mode="overwrite")



### TEAM files
# sample MIL,N,Milwaukee,Brewers
teamSchema = StructType([
    # prepended from filename
    StructField("year",StringType(),False), 

    # rest from record
    StructField("team_id",StringType(),False), 
    StructField("league",StringType(),False), 
    StructField("city",StringType(),False), 
    StructField("teamName",StringType(),False)
])
teamRowsRDD = sc.wholeTextFiles(teamPath).flatMap(
    getWholeTextParserWithPathTransform(lambda x:int(x[4:]))    
)
teamDataFrame = sqlContext.createDataFrame(teamRowsRDD, teamSchema)
teamDataFrame.registerTempTable("team")
teamDataFrame.write.save(DIST_PARQUET_DIR + "team", format="parquet", mode="overwrite")




print("done")

done


In [ ]:

sqlContext.sql("describe roster").show()

sql = """
    select year, count(*)
    from roster
    group by year
    order by year asc

"""

sqlContext.sql(sql).show(50)

In [ ]:
for t in sqlContext.tableNames():
    print(t)
    sqlContext.sql("DESCRIBE %s" % t).show()
    
    
    # TODO move this to a write everything step at the end
    # dfByType[recType].write.save(DIST_PARQUET_DIR + recType, format="parquet", mode="overwrite")

    
